In [94]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from dateutil.relativedelta import relativedelta
import calendar

In [336]:
# Necessary methods
def divideDateRange(rangeStart, rangeEnd, interval, count):
    """
    Divides a date range to specific intervals. The process starts from the end date and
    goes backwards to reach the start date.

    Args:
        rangeStart, rangeEnd: datetime.datetime: Start and end of the desired date range.
        interval: str: One of the following items [months, days, years]
        count: int: The length of each interval 
    
    Returns: 
        A list of dates.
    """
    lstOutput = []
    temp = rangeEnd
    while rangeStart < temp:
        lstOutput.append(temp)
        if interval == "days":
            temp -= relativedelta(days=count)
        elif interval == "months":
            temp -= relativedelta(months=count)
        elif interval == "years":
            temp -= relativedelta(years=count)
    
    return lstOutput

def barclayHedgeIndex(fileLoc):
    """
    Gets the Barclay Hedge Index data and returns it as a dataframe.
    """
    # Map the months to numbers
    mon = {month: index for index, month in enumerate(calendar.month_abbr) if month}

    # Read and process the data
    data = pd.read_excel(fileLoc, header=1)
    data.dropna(inplace=True)
    data.columns = ["year"]+ [str(i) for i in range(1,13)] + ["YTD"]
    
    tmpMonth = []
    tmpReturn = []
    for _, row in data.iterrows():
        for i in range(1,13):
            tmpMonth.append(datetime.datetime(year=int(row.year),month=i,day=1)+relativedelta(months=1))
            tmpReturn.append(float(row[str(i)]))
    
    df = pd.DataFrame(list(zip(tmpMonth,tmpReturn)), columns=["Date", "barclayReturn"])
    return df

def getMonthlyReturns(fundAuM):
    """
    Returns a dataframe containing monthly returns of the fund.

    Args:
        fundAuM: pd.Dataframe: A dataframe containing AuM of the fund at the end 
            of each day. SHould contain columns named ["Date", "AuM"]
    """
    endDate = fundAuM.iloc[-1].Date 
    startMonth = fundAuM.iloc[0].Date.month
    startYear = fundAuM.iloc[0].Date.year

    rangeStart = datetime.datetime(year = startYear, month=startMonth, day=1)
    rangeEnd = rangeStart
    monthReturn, monthStart, monthEnd = [], [], []

    while True:
        # Setting the difference between start and end of each month as its return
        rangeEnd += relativedelta(months=1)

        # Get the dataframe for each month
        tmpDataframe = fundAuM[(rangeStart <= fundAuM.Date)&(fundAuM.Date < rangeEnd)]

        # Calculate the return
        monthReturn.append((tmpDataframe.iloc[-1].AuM-tmpDataframe.iloc[0].AuM)/tmpDataframe.iloc[0].AuM)
        monthStart.append(rangeStart)
        monthEnd.append(rangeStart + relativedelta(months=1))

        if endDate <= rangeEnd:
            break
        rangeStart += relativedelta(months=1)

    fundMonthlyReturns = pd.DataFrame(list(zip(monthEnd,monthReturn)), columns=["Date", "fundReturn"])
    return fundMonthlyReturns

def calcCum(returns):
    """
    Calculates the cumulative return of a pandas series

    Args: 
        returns: pd.Series: A pandas series or a column
    
    Returns:
        A float number
    """
    __cumReturn = 1
    for ret in returns:
        __cumReturn = __cumReturn * (1 + np.float32(ret))
    
    return __cumReturn - 1

def calcCumReturnInRange(df, interval, count):
    """
    Calculates the cumulative return in a range based manner. 

    Args: 
        df: pd.Dataframe: A pandas dataframe containing monthly returns. 
            First columns should be end of the month in datetime, The rest 
            of the columns should be returns. This columns should be denoted 
            by "Date" label.
        interval: str: One of the following items [months, days, years]
        count: int: The length of each interval 
    
    Returns:
        A pandas dataframe
    """
    # Sorting the dataframe
    df = df.sort_values(["Date"], ascending=False, ignore_index=True)
    __lastDate, __firstDate = df.iloc[0].Date, df.iloc[-1].Date

    dateRange = divideDateRange(__firstDate, __lastDate, interval, count)
    
    # The output dataframe
    outDf = pd.DataFrame(columns=["intervalStart", "intervalEnd"] + list(df.columns)[1:])
    
    __tmpLst = []
    for i in range(len(dateRange)-1):
        __df = df[(dateRange[i+1]<=df.Date)&(df.Date<dateRange[i])]
        __tmpLst = []

        for column in __df:
            # Calculate cumulative return for each columns except Date
            if column == "Date": continue
            __tmpLst.append(calcCum(__df[column]))
        
        # Add the new row to the output dataframe
        outDf.loc[i] = [dateRange[i+1],dateRange[i]-relativedelta(days=1)] + __tmpLst

    return outDf

def calcCaptureRate(df):
    """
    Calculates the capture rate of fund relative to benchmark.

    Args: 
        df: pd.Dataframe: A dataframe containing two columns. First the 
            fund returns and the second, the benchmark returns.
    """
    numerator, denumerator = 1, 1
    numerator = (df.iloc[:,0]+1).product(axis = 0)
    denumerator = (df.iloc[:,1]+1).product(axis = 0)
    
    return (numerator - 1) / (denumerator - 1)

def netGrowthRate(AuMLastDay, AuMFirstDay, annualized):
    """
    Returns the net growth rate of the fund's AuM 
    The formula: (Last day value - First day value)/First day value = growth rate

    Args:
        AuMLastDay, AuMFirstDay: float: AuM of the fund at the beginning and end of the interval
    
    Returns: 
        float.
    """
    if annualized:
        return np.power((AuMLastDay-AuMFirstDay)/AuMFirstDay + 1, 1/3) - 1
    else:
        return (AuMLastDay-AuMFirstDay)/AuMFirstDay

In [91]:
# Data and super variables
# Import the data for fund 1
f1_DailyAuM = pd.read_csv("./Data/Daily AuM - AuM (1).csv")
f1_DailyTrades = pd.read_csv("./Data/Daily AuM - Trades_10.csv")

# Get the union of both datasets (Date-wise)
f1_DailyAuM.Date = pd.to_datetime(f1_DailyAuM.Date, dayfirst = True, format = "mixed")
f1_DailyTrades.buy_date = pd.to_datetime(f1_DailyTrades.buy_date, dayfirst = True, format = "mixed")
f1_DailyTrades.sell_date = pd.to_datetime(f1_DailyTrades.sell_date, dayfirst = True, format = "mixed")

f1_DailyTrades_LastDate = f1_DailyTrades.iloc[-1].sell_date
f1_DailyAuM_LastDate = f1_DailyAuM.iloc[-1].Date

if f1_DailyTrades_LastDate < f1_DailyAuM_LastDate:
    f1_DailyAuM = f1_DailyAuM[f1_DailyAuM.Date <= f1_DailyTrades_LastDate]
else:
    f1_DailyTrades = f1_DailyTrades[f1_DailyTrades.sell_date <= f1_DailyAuM_LastDate]

# start and end of date of the entire dataset
START_DATE, END_DATE = f1_DailyAuM.iloc[0].Date, f1_DailyAuM.iloc[-1].Date 
START_MONTH, END_MONTH = f1_DailyAuM.iloc[0].Date.month, f1_DailyAuM.iloc[-1].Date.month 
START_YEAR, END_YEAR = f1_DailyAuM.iloc[0].Date.year, f1_DailyAuM.iloc[-1].Date.year 

print(f"The entire date range is from {START_DATE} to {END_DATE} ({(f1_DailyAuM.iloc[-1].Date - f1_DailyAuM.iloc[0].Date)})")


The entire date range is from 2019-12-27 00:00:00 to 2023-10-31 00:00:00 (1404 days 00:00:00)


In [92]:
# Growth rate of fund net worth
# This metric is calculated on three intervals, six months, 1 year and 3 years
range_6Mo = divideDateRange(START_DATE, END_DATE, datetime.timedelta(days=30*6))
range_1Yr = divideDateRange(START_DATE, END_DATE, datetime.timedelta(days=30*12))
range_3Yrs = divideDateRange(START_DATE, END_DATE, datetime.timedelta(days=30*12*3))

tmpAuM_6Mo = f1_DailyAuM[(range_6Mo[1]<f1_DailyAuM.Date)&(f1_DailyAuM.Date<=range_6Mo[0])]
growthRate_6Mo = netGrowthRate(tmpAuM_6Mo.iloc[-1].AuM, tmpAuM_6Mo.iloc[0].AuM, False)

tmpAuM_1Yr = f1_DailyAuM[(range_1Yr[1]<f1_DailyAuM.Date)&(f1_DailyAuM.Date<=range_1Yr[0])]
growthRate_1Yr = netGrowthRate(tmpAuM_1Yr.iloc[-1].AuM, tmpAuM_1Yr.iloc[0].AuM, False)

tmpAuM_3Yrs = f1_DailyAuM[(range_3Yrs[1]<f1_DailyAuM.Date)&(f1_DailyAuM.Date<=range_3Yrs[0])]
growthRate_3Yrs = netGrowthRate(tmpAuM_3Yrs.iloc[-1].AuM, tmpAuM_3Yrs.iloc[0].AuM, False)

print(f"6 months growth rate (%) = {growthRate_6Mo*100} calculated from {tmpAuM_6Mo.iloc[-1].Date} to {tmpAuM_6Mo.iloc[0].Date}")
print(f"1 year growth rate (%) = {growthRate_1Yr*100} calculated from {tmpAuM_1Yr.iloc[-1].Date} to {tmpAuM_1Yr.iloc[0].Date}")
print(f"3 years growth rate (%) = {growthRate_3Yrs*100} calculated from {tmpAuM_3Yrs.iloc[-1].Date} to {tmpAuM_3Yrs.iloc[0].Date}")


6 months growth rate (%) = 4.3824640703169635 calculated from 2023-10-31 00:00:00 to 2023-05-05 00:00:00
1 year growth rate (%) = 49.44325780125048 calculated from 2023-10-31 00:00:00 to 2022-11-06 00:00:00
3 years growth rate (%) = 986.9440892651639 calculated from 2023-10-31 00:00:00 to 2020-11-16 00:00:00


In [337]:
# Up/Down capture return
# Getting the monthly return of the Fund's AuM
fundReturns = getMonthlyReturns(f1_DailyAuM)
barclayReturns = barclayHedgeIndex("./Data/Barclay_Hedge_Fund_Indexhistorical_data.xls")
barclayReturns = barclayReturns[(fundReturns.iloc[0].Date <= barclayReturns.Date)&(barclayReturns.Date <= fundReturns.iloc[-1].Date)].reset_index(drop=True)

df = pd.concat([fundReturns.Date, fundReturns.fundReturn, barclayReturns.barclayReturn], axis = 1)
dfCumulativeReturn = calcCumReturnInRange(df, "months", 6)

# Up capture
upDf = dfCumulativeReturn[0 <= dfCumulativeReturn.barclayReturn]
upCapture = calcCaptureRate(upDf[["fundReturn", "barclayReturn"]])
upCapture
# Down capture
dnDf = dfCumulativeReturn[dfCumulativeReturn.barclayReturn <= 0]
dnCapture = calcCaptureRate(dnDf[["fundReturn", "barclayReturn"]])
dnCapture

5.657128645083438 1.4775264087398712
1.735177591639733 0.8987804992108044


-7.2632011214009795